In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm

In [2]:
data = pd.read_csv("dataset-logistic-regression.csv")

y = data["y"].astype(float).to_numpy()
X = data.drop(columns=["y"]).to_numpy()

n, p = X.shape
print("n, p =", n, p)

n, p = 10000 100


In [3]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def f(beta, X, y):
    p = sigmoid(X @ beta)
    # clamp to avoid log(0)
    p = np.clip(p, 1e-8, 1 - 1e-8)
    return -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

def grad_f(beta, X, y):
    p = sigmoid(X @ beta)
    return X.T @ (p - y)

In [4]:
glm_model = sm.Logit(y, X)
glm_result = glm_model.fit(disp=False)

beta_glm = glm_result.params
print("tail(glm coefficients):")
print(beta_glm[-6:])

tail(glm coefficients):
[-0.04860354  0.03384919  0.14973405  0.17026521  0.01979437  0.00700259]


In [5]:
def gradient_descent_backtracking(X, y, beta_init=None, tol=1e-6, max_iter=10000, epsilon=0.5, tau=0.8):
    n, p = X.shape
    beta = np.zeros(p) if beta_init is None else beta_init.copy()
    eta = 1.0
    iterations = 0

    for _ in range(max_iter):
        iterations += 1
        grad = grad_f(beta, X, y)
        eta_bt = eta
        beta_new = beta - eta_bt * grad

        while f(beta_new, X, y) > f(beta, X, y) - epsilon * eta_bt * np.sum(grad**2):
            eta_bt *= tau
            beta_new = beta - eta_bt * grad

        if np.linalg.norm(beta_new - beta) < tol:
            beta = beta_new
            break

        beta = beta_new

    return {"beta": beta, "iterations": iterations}

gd_back = gradient_descent_backtracking(X, y)
print("tail(gd_back beta):")
print(gd_back["beta"][-6:])

/var/folders/t0/cl4mx7656030fy5dm2t2jwsr0000gn/T/ipykernel_36632/2326215551.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


tail(gd_back beta):
[-0.04860114  0.03385028  0.14973027  0.17026175  0.01979193  0.00700404]


In [6]:
def gradient_descent_nesterov(X, y, beta_init=None, tol=1e-6, max_iter=10000, epsilon=0.5, tau=0.8):
    n, p = X.shape
    beta = np.zeros(p) if beta_init is None else beta_init.copy()
    beta_prev = beta.copy()
    eta = 1.0
    iterations = 0

    for it in range(1, max_iter + 1):
        iterations += 1

        momentum_step = beta + (it - 1) / (it + 2) * (beta - beta_prev)
        grad = grad_f(momentum_step, X, y)

        eta_bt = eta
        beta_new = momentum_step - eta_bt * grad

        while f(beta_new, X, y) > f(momentum_step, X, y) - epsilon * eta_bt * np.sum(grad**2):
            eta_bt *= tau
            beta_new = momentum_step - eta_bt * grad

        if np.linalg.norm(beta_new - beta) < tol:
            beta = beta_new
            break

        beta_prev = beta
        beta = beta_new

    return {"beta": beta, "iterations": iterations}

gd_nest = gradient_descent_nesterov(X, y)
print("tail(gd_nest beta):")
print(gd_nest["beta"][-6:])

/var/folders/t0/cl4mx7656030fy5dm2t2jwsr0000gn/T/ipykernel_36632/2326215551.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


tail(gd_nest beta):
[-0.04860551  0.03384797  0.14973736  0.17026769  0.01979698  0.00700183]


In [7]:
def amsgrad_adam(X, y, beta_init=None, eta=0.01, beta1=0.9, beta2=0.999, eps=1e-8, max_iter=1000, tol=1e-6):
    n, p = X.shape
    beta = np.zeros(p) if beta_init is None else beta_init.copy()

    m = np.zeros(p)
    z = np.full(p, eps)       # second moment
    z_hat = np.full(p, eps)   # max past second moments (AMSGrad)
    iterations = 0

    for t in range(1, max_iter + 1):
        iterations += 1
        grad = grad_f(beta, X, y)

        m = beta1 * m + (1 - beta1) * grad
        z = beta2 * z + (1 - beta2) * (grad**2)
        z_hat = np.maximum(z_hat, z)

        m_hat = m / (1 - beta1**t)
        z_tilda = 1.0 / (np.sqrt(z_hat) + eps)

        beta_new = beta - eta * (z_tilda * m_hat)

        if np.linalg.norm(beta_new - beta) < tol:
            beta = beta_new
            break

        beta = beta_new

    return {"beta": beta, "iterations": iterations}

adam = amsgrad_adam(X, y)
print("tail(adam beta):")
print(adam["beta"][-6:])

tail(adam beta):
[-0.04860553  0.03384812  0.14973647  0.17026696  0.01979628  0.00700074]


In [8]:
def stochastic_gd_fixed_stepsize(X, y, beta_init=None, tol=1e-6, max_iter=10000, c=1.0, alpha=0.5, batch_size=1, seed=None):
    rng = np.random.default_rng(seed)
    n, p = X.shape
    beta = np.zeros(p) if beta_init is None else beta_init.copy()
    iterations = 0

    for it in range(1, max_iter + 1):
        iterations += 1
        eta = c / (it ** alpha)

        batch_idx = rng.choice(n, size=batch_size, replace=False)
        Xb = X[batch_idx, :]
        yb = y[batch_idx]

        grad = grad_f(beta, Xb, yb)
        beta_new = beta - eta * grad

        if np.linalg.norm(beta_new - beta) < tol:
            beta = beta_new
            break

        beta = beta_new

    return {"beta": beta, "iterations": iterations}

gd_sto = stochastic_gd_fixed_stepsize(X, y)
print("tail(gd_sto beta):")
print(gd_sto["beta"][-6:])

tail(gd_sto beta):
[0.52192073 0.54120334 1.12626663 0.57923585 0.49724461 0.53950405]


In [9]:
def stochastic_gd_AMS(X, y, beta_init=None, tol=1e-6, max_iter=10000,
                      batch_size=1, eta=0.01, eps=1e-8, beta1=0.9, beta2=0.999, seed=None):

    rng = np.random.default_rng(seed)
    n, p = X.shape
    beta = np.zeros(p) if beta_init is None else beta_init.copy()
    iterations = 0

    m = np.zeros(p)
    z = np.full(p, eps)
    z_hat = np.full(p, eps)

    for it in range(1, max_iter + 1):
        iterations += 1

        batch_idx = rng.choice(n, size=batch_size, replace=False)
        Xb = X[batch_idx, :]
        yb = y[batch_idx]

        grad = grad_f(beta, Xb, yb)

        m = beta1 * m + (1 - beta1) * grad
        z = beta2 * z + (1 - beta2) * (grad**2)
        z_hat = np.maximum(z_hat, z)

        m_hat = m / (1 - beta1**it)
        z_tilda = 1.0 / (np.sqrt(z_hat) + eps)

        beta_new = beta - eta * (z_tilda * m_hat)

        if np.linalg.norm(beta_new - beta) < tol:
            beta = beta_new
            break

        beta = beta_new

    return {"beta": beta, "iterations": iterations}

gd_sto_AMS = stochastic_gd_AMS(X, y)
print("tail(gd_sto_AMS beta):")
print(gd_sto_AMS["beta"][-6:])

tail(gd_sto_AMS beta):
[-0.12226314  0.13915059  0.21244903  0.26104846  0.19860774 -0.04042662]


In [ ]:
gd_sto_100 = stochastic_gd_fixed_stepsize(X, y, batch_size=100)
gd_sto_200 = stochastic_gd_fixed_stepsize(X, y, batch_size=200)
gd_sto_500 = stochastic_gd_fixed_stepsize(X, y, batch_size=500)

gd_sto_AMS_100 = stochastic_gd_AMS(X, y, batch_size=100)
gd_sto_AMS_200 = stochastic_gd_AMS(X, y, batch_size=200)
gd_sto_AMS_500 = stochastic_gd_AMS(X, y, batch_size=500)

/var/folders/t0/cl4mx7656030fy5dm2t2jwsr0000gn/T/ipykernel_36632/2326215551.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [ ]:
def est_error(beta_hat, beta_ref):
    return np.linalg.norm(beta_hat - beta_ref)

error_gd_back = est_error(gd_back["beta"], beta_glm)
error_gd_nest = est_error(gd_nest["beta"], beta_glm)
error_adam = est_error(adam["beta"], beta_glm)

error_gd_sto_100 = est_error(gd_sto_100["beta"], beta_glm)
error_gd_sto_200 = est_error(gd_sto_200["beta"], beta_glm)
error_gd_sto_500 = est_error(gd_sto_500["beta"], beta_glm)

error_gd_sto_AMS_100 = est_error(gd_sto_AMS_100["beta"], beta_glm)
error_gd_sto_AMS_200 = est_error(gd_sto_AMS_200["beta"], beta_glm)
error_gd_sto_AMS_500 = est_error(gd_sto_AMS_500["beta"], beta_glm)

In [ ]:
error_df = pd.DataFrame({
    "method": [
        "gd_back",
        "gd_nest",
        "adam",
        "gd_sto_100",
        "gd_sto_200",
        "gd_sto_500",
        "gd_sto_AMS_100",
        "gd_sto_AMS_200",
        "gd_sto_AMS_500",
    ],
    "estimation_error": [
        error_gd_back,
        error_gd_nest,
        error_adam,
        error_gd_sto_100,
        error_gd_sto_200,
        error_gd_sto_500,
        error_gd_sto_AMS_100,
        error_gd_sto_AMS_200,
        error_gd_sto_AMS_500,
    ],
    "iterations": [
        gd_back["iterations"],
        gd_nest["iterations"],
        adam["iterations"],
        gd_sto_100["iterations"],
        gd_sto_200["iterations"],
        gd_sto_500["iterations"],
        gd_sto_AMS_100["iterations"],
        gd_sto_AMS_200["iterations"],
        gd_sto_AMS_500["iterations"],
    ]
})

error_df

,method,estimation_error,iterations
0,gd_back,0.000022,137
1,gd_nest,0.000016,488
2,adam,0.000014,244
3,gd_sto_100,1.468215,10000
4,gd_sto_200,5.271125,10000
5,gd_sto_500,9.668076,10000
6,gd_sto_AMS_100,0.320782,10000
7,gd_sto_AMS_200,0.273686,10000
8,gd_sto_AMS_500,0.199711,10000
